In [234]:
import requests     
import json         
import pandas as pd 
from bs4 import BeautifulSoup
import json
import re
import csv
from io import StringIO
from datetime import datetime
import numpy as np
import time
import warnings
warnings.filterwarnings("ignore")
import os

In [235]:
os.chdir('C:\\Users\\fizpo')

In [236]:
Transactions = pd.read_csv('Transactions.csv')

In [237]:
Transactions.head(n = 2)

,Data,Czas,Produkt,ISIN,Giełda referenc,Miejsce wykonania,Liczba,Kurs,Unnamed: 8,Wartość lokalna,Unnamed: 10,Wartość,Unnamed: 12,Kurs wymian,Opłata transakcyjna,Unnamed: 15,Razem,Unnamed: 17,Identyfikator zlecenia
0,02-02-2022,17:31,PAYPAL HOLDINGS INC.,US70450Y1038,NDQ,CDED,1,131.75,USD,-131.75,USD,-116.63,EUR,1.1296,-0.5,EUR,-117.13,EUR,b9d097fb-1969-49dc-ab52-6e9ed379444e
1,04-01-2022,20:37,APPLE INC. - COMMON ST,US0378331005,NDQ,CDED,1,179.80,USD,-179.80,USD,-159.24,EUR,1.1291,-0.5,EUR,-159.74,EUR,5a614ead-62b9-47f8-b045-d15a7c14c99d


In [238]:
exchCode = [None]*len(Transactions)
Transactions['exchCode'] = exchCode
Transactions = Transactions.rename({'Unnamed: 8':'waluta_pozycji','Unnamed: 10':'waluta_wartości_lok','Unnamed: 12':'waluta_zak','Unnamed: 15':'Opł_trans_waluta','Unnamed: 17':'waluta_zak_razem'}, axis= 1)
Transactions['Data'] = Transactions['Data'].astype(str) + ' ' + Transactions['Czas'].astype(str)
Transactions['Data'] = pd.to_datetime(Transactions['Data'],dayfirst=True)
Transactions = Transactions.sort_values('Data').reset_index(drop=True) 

In [239]:
Transactions.head()

,Data,Czas,Produkt,ISIN,Giełda referenc,Miejsce wykonania,Liczba,Kurs,waluta_pozycji,Wartość lokalna,waluta_wartości_lok,Wartość,waluta_zak,Kurs wymian,Opłata transakcyjna,Opł_trans_waluta,Razem,waluta_zak_razem,Identyfikator zlecenia,exchCode
0,2020-03-18 09:02:00,09:02,VANGUARD S&P500,IE00B3XXRP09,LSE,XLON,1,38.000,GBP,-38.00,GBP,-41.67,EUR,0.9119,-2.01,EUR,-43.68,EUR,1bfea344-4165-47d2-9820-13c6384d2da7,None
1,2020-03-26 16:33:00,16:33,BANK OF AMERICA CORPOR,US0605051046,NSY,CDED,1,22.445,USD,-22.45,USD,-20.39,EUR,1.1010,-0.50,EUR,-20.89,EUR,18aead73-a12f-4fbc-bae9-419933f647a2,None
2,2020-03-26 17:38:00,17:38,DELTA AIR LINES INC.,US2473617023,NSY,XNAS,1,35.000,USD,-35.00,USD,-31.75,EUR,1.1025,-0.50,EUR,-32.25,EUR,0746aed0-09bd-4a5f-8dd6-4a5d2ad2d854,None
3,2020-03-30 15:30:00,15:30,DELTA AIR LINES INC.,US2473617023,NSY,XNYS,1,28.110,USD,-28.11,USD,-25.49,EUR,1.1028,-0.50,EUR,-25.99,EUR,682ce8ec-0dc5-439c-b9ab-fcfbadb53e22,None
4,2020-03-31 14:01:00,14:01,ISHRC S&P 500,IE00B5BMR087,LSE,XLON,1,260.000,USD,-260.00,USD,-237.33,EUR,1.0955,-2.07,EUR,-239.40,EUR,27dcf306-9e81-4c38-bc9e-1ed5099a9a4d,None


In [240]:
for i in range(len(Transactions)):
    if 'NON TRADEABLE' in Transactions['Produkt'][i] or (Transactions['Liczba'][i] == 0):
        print(Transactions['Produkt'][i])
        Transactions.drop(i,inplace = True)
Transactions = Transactions.reset_index(drop=True) 

HOTH THERAPEUTICS CM
TARONIS TECHS INC.DL-,001 - NON TRADEABLE


In [241]:
Transactions['ISIN'].unique()

array(['IE00B3XXRP09', 'US0605051046', 'US2473617023', 'IE00B5BMR087',
       'US0088751062', 'US00848K1016', 'US8762142060', 'US0400476075',
       'US80105N1054', 'US3755581036', 'US9216591084', 'US5949181045',
       'US30303M1027', 'US89151E1091', 'US30231G1022', 'US4581401001',
       'US4781601046', 'US0378331005', 'US9311421039', 'US46625H1005',
       'US92826C8394', 'US01609W1027', 'US88034P1093', 'NL0009805522',
       'US16939P1066', 'US92343X1000', 'US0846707026', 'US34959E1091',
       'US78781P1057', 'IL0011691438', 'US7427181091', 'US70450Y1038',
       'US87105L1044', 'US79466L3024', 'US67066G1040', 'US65339F1012'],
      dtype=object)

In [242]:
Transactions['Giełda referenc'].unique()

array(['LSE', 'NSY', 'NDQ', 'ASE'], dtype=object)

In [243]:
closed_pos = pd.DataFrame()
current_pos = pd.DataFrame()
for produkt in (Transactions['Produkt'].unique()):
    k = Transactions[Transactions['Produkt'] == produkt]
    s = 0
    for i in range (len(k)):
        s += k.iloc[i][6]
    if s == 0:
        closed_pos = closed_pos.append(k,ignore_index = True)
    else:
        current_pos = current_pos.append(k,ignore_index = True)
    

In [244]:
items_closed = list(zip(closed_pos['ISIN'],closed_pos['Giełda referenc'], closed_pos['exchCode'],closed_pos['waluta_pozycji']))
items_current = list(zip(current_pos['ISIN'],current_pos['Giełda referenc'], current_pos['exchCode'],current_pos['waluta_pozycji']))

In [245]:
stocks_closed = []
for item in items_closed:
    stock = list(item)
    stocks_closed.append(stock)

In [246]:
stocks_current = []
for item in items_current:
    stock = list(item)
    stocks_current.append(stock)

In [247]:
for i in range (len(stocks_closed)):
    if stocks_closed[i][1] in ('NDQ', 'NSY','ASE'):
        stocks_closed[i][2] = 'US'
    elif stocks_closed[i][1] == 'LSE':
        stocks_closed[i][2] = 'LN'
    elif stocks_closed[i][0] == 'US8762142060' or stocks[i][1] == 'DEG' :
        stocks_closed[i][2] = 'US'
        stocks_closed[i][3] = 'USD'

In [248]:
for i in range (len(stocks_current)):
    if stocks_current[i][1] in ('NDQ', 'NSY','ASE'):
        stocks_current[i][2] = 'US'
    elif stocks_current[i][1] == 'LSE':
        stocks_current[i][2] = 'LN'
    elif stocks_current[i][0] == 'US8762142060' or stocks[i][1] == 'DEG' :
        stocks_current[i][2] = 'US'
        stocks_current[i][3] = 'USD'

In [249]:
jobs_closed = []
for i in range (len(stocks_closed)):
    job = {'idType': 'ID_ISIN', 'idValue': stocks_closed[i][0], 'exchCode': stocks_closed[i][2], 'currency': stocks_closed[i][3]}
    jobs_closed.append(job)

In [250]:
jobs_current = []
for i in range (len(stocks_current)):
    job = {'idType': 'ID_ISIN', 'idValue': stocks_current[i][0], 'exchCode': stocks_current[i][2], 'currency': stocks_current[i][3]}
    jobs_current.append(job)

In [251]:
openfigi_apikey = '844b2af4-4c56-455c-907d-3fc4b5226880'  
openfigi_url = 'https://api.openfigi.com/v2/mapping'
openfigi_headers = {'Content-Type': 'text/json'}

In [252]:
def map_jobs_closed(jobs_closed):
    if openfigi_apikey:
        openfigi_headers['X-OPENFIGI-APIKEY'] = openfigi_apikey
    response = requests.post(url=openfigi_url, headers=openfigi_headers,
                             json=jobs_closed)
    if response.status_code != 200:
        raise Exception('Bad response code {}'.format(str(response.status_code)))
    return response.json()
job_results_closed = map_jobs_closed(jobs_closed)

In [256]:
closed_pos

,Data,Czas,Produkt,ISIN,Giełda referenc,Miejsce wykonania,Liczba,Kurs,waluta_pozycji,Wartość lokalna,waluta_wartości_lok,Wartość,waluta_zak,Kurs wymian,Opłata transakcyjna,Opł_trans_waluta,Razem,waluta_zak_razem,Identyfikator zlecenia,exchCode
0,2020-03-18 09:02:00,09:02,VANGUARD S&P500,IE00B3XXRP09,LSE,XLON,1,38.0000,GBP,-38.00,GBP,-41.67,EUR,0.9119,-2.01,EUR,-43.68,EUR,1bfea344-4165-47d2-9820-13c6384d2da7,None
1,2020-06-12 16:26:00,16:26,VANGUARD S&P500,IE00B3XXRP09,LSE,XLON,-1,46.2175,GBP,46.22,GBP,51.48,EUR,0.8978,-2.02,EUR,49.46,EUR,0bcdc682-bf9a-4716-9f47-3accb42c91d3,None
2,2020-03-26 16:33:00,16:33,BANK OF AMERICA CORPOR,US0605051046,NSY,CDED,1,22.4450,USD,-22.45,USD,-20.39,EUR,1.1010,-0.50,EUR,-20.89,EUR,18aead73-a12f-4fbc-bae9-419933f647a2,None
3,2020-04-28 17:15:00,17:15,BANK OF AMERICA CORPOR,US0605051046,NSY,CDED,-1,24.1500,USD,24.15,USD,22.28,EUR,1.0838,-0.50,EUR,21.78,EUR,7d9ea12e-b6bb-4402-8d1a-5845daac8702,None
4,2020-03-26 17:38:00,17:38,DELTA AIR LINES INC.,US2473617023,NSY,XNAS,1,35.0000,USD,-35.00,USD,-31.75,EUR,1.1025,-0.50,EUR,-32.25,EUR,0746aed0-09bd-4a5f-8dd6-4a5d2ad2d854,None
5,2020-03-30 15:30:00,15:30,DELTA AIR LINES INC.,US2473617023,NSY,XNYS,1,28.1100,USD,-28.11,USD,-25.49,EUR,1.1028,-0.50,EUR,-25.99,EUR,682ce8ec-0dc5-439c-b9ab-fcfbadb53e22,None
6,2020-11-09 16:56:00,16:56,DELTA AIR LINES INC.,US2473617023,NSY,SOHO,-2,35.1810,USD,70.36,USD,59.53,EUR,1.1820,-0.51,EUR,59.02,EUR,aec7e667-d1e4-45ed-81ae-fa361070a9f1,None
7,2020-03-31 14:01:00,14:01,ISHRC S&P 500,IE00B5BMR087,LSE,XLON,1,260.0000,USD,-260.00,USD,-237.33,EUR,1.0955,-2.07,EUR,-239.40,EUR,27dcf306-9e81-4c38-bc9e-1ed5099a9a4d,None
8,2020-06-12 16:27:00,16:27,ISHRC S&P 500,IE00B5BMR087,LSE,CHIX,-1,307.0100,USD,307.01,USD,271.96,EUR,1.1289,-2.08,EUR,269.88,EUR,eb6aa768-a02a-45ae-8d94-b75352942b1b,None
9,2020-04-14 18:28:00,18:28,AIKIDO PHARMA INC,US0088751062,NDQ,XNAS,28,1.0600,USD,-29.68,USD,-27.05,EUR,1.0972,-0.60,EUR,-27.65,EUR,b2099809-9bce-4f05-bddb-279930d80317,None


In [254]:
job_results_closed

[{'data': [{'figi': 'BBG003241JV5',
    'name': 'VANG S&P500 USDD',
    'ticker': 'VUSA',
    'exchCode': 'LN',
    'compositeFIGI': 'BBG003241JT8',
    'uniqueID': None,
    'securityType': 'ETP',
    'marketSector': 'Equity',
    'shareClassFIGI': 'BBG003241JZ1',
    'uniqueIDFutOpt': None,
    'securityType2': 'Mutual Fund',
    'securityDescription': 'VUSA'}]},
 {'data': [{'figi': 'BBG003241JV5',
    'name': 'VANG S&P500 USDD',
    'ticker': 'VUSA',
    'exchCode': 'LN',
    'compositeFIGI': 'BBG003241JT8',
    'uniqueID': None,
    'securityType': 'ETP',
    'marketSector': 'Equity',
    'shareClassFIGI': 'BBG003241JZ1',
    'uniqueIDFutOpt': None,
    'securityType2': 'Mutual Fund',
    'securityDescription': 'VUSA'}]},
 {'data': [{'figi': 'BBG000BCTLF6',
    'name': 'BANK OF AMERICA CORP',
    'ticker': 'BAC',
    'exchCode': 'US',
    'compositeFIGI': 'BBG000BCTLF6',
    'uniqueID': None,
    'securityType': 'Common Stock',
    'marketSector': 'Equity',
    'shareClassFIGI': 'B

In [255]:
stocks_closed = [d['data'][0] for d in job_results_closed]

KeyError: 'data'

In [228]:
stocks_closed = pd.DataFrame.from_dict(stocks_closed)

In [196]:
def map_jobs_current(jobs_current):
    if openfigi_apikey:
        openfigi_headers['X-OPENFIGI-APIKEY'] = openfigi_apikey
    response = requests.post(url=openfigi_url, headers=openfigi_headers,
                             json=jobs_current)
    if response.status_code != 200:
        raise Exception('Bad response code {}'.format(str(response.status_code)))
    return response.json()
job_results_current = map_jobs_current(jobs_current)

In [197]:
stocks_current = [d['data'][0] for d in job_results_current]

In [198]:
stocks_current = pd.DataFrame.from_dict(stocks_current)

In [35]:
d_closed = pd.DataFrame(list(zip(closed_pos['Produkt'].unique(),stocks_closed['name'].unique(),stocks_closed['ticker'].unique())), columns=['degiro_name','figi_name','ticker'])
currency = [None]*len(d_closed)
pair_currency = [None]*len(d_closed)
d_closed['currency'] = currency
d_closed['pair_currency'] = pair_currency

KeyError: 'name'

In [36]:
d_current = pd.DataFrame(list(zip(current_pos['Produkt'].unique(),stocks_current['name'].unique(),stocks_current['ticker'].unique())), columns=['degiro_name','figi_name','ticker'])
currency = [None]*len(d_current)
pair_currency = [None]*len(d_current)
d_current['currency'] = currency
d_current['pair_currency'] = pair_currency

In [37]:
for i in range (len(d_closed)):
    for j in range (len(closed_pos['Produkt'])):
        if closed_pos['Produkt'][j] == d_closed['degiro_name'][i]:
            d_closed['currency'][i] = closed_pos['waluta_pozycji'][j]
            
for i in range (len(d_closed)):
    if d_closed['currency'][i] == 'USD':
        d_closed['pair_currency'][i] = 'EUR=X'
    elif d_closed['currency'][i] == 'GBP':
        d_closed['pair_currency'][i] = 'GBPEUR=X'
    if d_closed['ticker'][i] == 'CSPX':
        d_closed['ticker'][i] = 'CSPX.L'
    if d_closed['ticker'][i] == 'VUSA':
        d_closed['ticker'][i] = 'VUSA.L'

NameError: name 'd_closed' is not defined

In [38]:
d_closed

NameError: name 'd_closed' is not defined

In [39]:
for i in range (len(d_current)):
    for j in range (len(current_pos['Produkt'])):
        if current_pos['Produkt'][j] == d_current['degiro_name'][i]:
            d_current['currency'][i] = current_pos['waluta_pozycji'][j]
            
for i in range (len(d_current)):
    if d_current['currency'][i] == 'USD':
        d_current['pair_currency'][i] = 'EUR=X'
    elif d_current['currency'][i] == 'GBP':
        d_current['pair_currency'][i] = 'GBPEUR=X'
    if d_current['ticker'][i] == 'BRK/B':
        d_current['ticker'][i] = 'BRK-B'


In [40]:
d_current

,degiro_name,figi_name,ticker,currency,pair_currency
0,GILEAD SCIENCES INC.,GILEAD SCIENCES INC,GILD,USD,EUR=X
1,MICROSOFT CORPORATION,MICROSOFT CORP,MSFT,USD,EUR=X
2,META PLATFORMS INC,META PLATFORMS INC-CLASS A,FB,USD,EUR=X
3,TOTAL S.A.,TOTALENERGIES SE -SPON ADR,TTE,USD,EUR=X
4,EXXON MOBIL CORPORATIO,EXXON MOBIL CORP,XOM,USD,EUR=X
5,INTEL CORPORATION - CO,INTEL CORP,INTC,USD,EUR=X
6,JOHNSON & JOHNSON COMM,JOHNSON & JOHNSON,JNJ,USD,EUR=X
7,APPLE INC. - COMMON ST,APPLE INC,AAPL,USD,EUR=X
8,WAL-MART STORES INC.,WALMART INC,WMT,USD,EUR=X
9,JP MORGAN CHASE & CO.,JPMORGAN CHASE & CO,JPM,USD,EUR=X


In [41]:
def parse_stock(stock,period1,period2):
    stock_url = 'https://query1.finance.yahoo.com/v7/finance/download/{}?'
    params = {
        'period1' : period1 ,
        'period2' : period2,
        'interval' : '1d',
        'events' : 'history'
    }
    response_stock = requests.get(stock_url.format(stock), params = params,headers={'User-agent': 'Mozilla/5.0'})
    file_stock = StringIO(response_stock.text)
    reader_stock = csv.reader(file_stock)
    data_stock = pd.DataFrame (reader_stock, columns = ['Data','Open','High','Low','Close','Adj Close','Volume']).drop([0])
    data_stock['Data'] = pd.to_datetime(data_stock['Data'], yearfirst=True)
    data_stock['Adj Close'] = pd.to_numeric(data_stock['Adj Close'])
    data_stock['Close'] = pd.to_numeric(data_stock['Close'])
    return data_stock

In [42]:
def parse_currency(currency,period1,period2):
    currency_url = 'https://query1.finance.yahoo.com/v7/finance/download/{}?'
    params = {
        'period1' : period1 ,
        'period2' : period2,
        'interval' : '1d',
        'events' : 'history'
    }
    response_currency = requests.get(currency_url.format(currency), params = params,headers={'User-agent': 'Mozilla/5.0'})
    file_currency = StringIO(response_currency.text)
    reader_currency = csv.reader(file_currency)
    fx = pd.DataFrame (reader_currency, columns = ['Data','Open','High','Low','Close','Adj Close','Volume']).drop([0])
    fx['Data'] = pd.to_datetime(fx['Data'], yearfirst=True)
    fx['Adj Close'] = pd.to_numeric(fx['Adj Close'])
    fx['Close'] = pd.to_numeric(fx['Close'])
    return fx

In [43]:
def profit_closed(merge, stock_degiro,for_zero):
    amount_of_bought_stocks = 0
    amount_of_sold_stocks = 0
    sum_of_bought_stocks = 0
    sum_of_sold_stocks = 0
    opłaty_trans = 0
    sum_of_bought_stocks_without_currency = 0
    sum_of_sold_stocks_without_currency = 0
    n = []
    roi_brutto_closed = [0]*for_zero
    for i in range (len(merge)):
        for j in range(len(stock_degiro)):
            if merge.iloc[i][0].date() == stock_degiro.iloc[j][0].date() and stock_degiro.iloc[j,6] > 0 :
                amount_of_bought_stocks += stock_degiro.iloc[j,6]
                sum_of_bought_stocks += (-1)*stock_degiro.iloc[j,11]
                sum_of_bought_stocks_without_currency += stock_degiro.iloc[j,7]
                opłaty_trans += (-1)*stock_degiro.iloc[j,14]
                a = -stock_degiro.iloc[j,11]/stock_degiro.iloc[j,6]
                for k in range(stock_degiro.iloc[j,6]):
                    n.append(a)
                average_price = sum_of_bought_stocks/amount_of_bought_stocks 
                average_price_without_currency = sum_of_bought_stocks_without_currency / amount_of_bought_stocks
            if data_stock.iloc[i][0].date() == stock_degiro.iloc[j][0].date() and stock_degiro.iloc[j,6] < 0:
                amount_of_sold_stocks += stock_degiro.iloc[j,6]
                sum_of_sold_stocks = stock_degiro.iloc[j,11]
                sum_of_sold_stocks_without_currency = stock_degiro.iloc[j,7]
                opłaty_trans += (-1)*stock_degiro.iloc[j,14]
                for l in range((-1)*stock_degiro.iloc[j,6]):
                    n.pop(0)
                if len(n) != 0:
                    average_price = sum(n)/len(n)
                else:
                    gains = ((-1)*sum_of_sold_stocks/amount_of_sold_stocks - average_price)/average_price
                    gains_without_currency = ((merge.iloc[i,1]) - average_price_without_currency)/average_price_without_currency
                    qqq = [gains_without_currency,merge.iloc[i][0].date()]
                    roi_brutto_closed.append(gains)
                    return roi_brutto_closed


        gains = ((merge.iloc[i,1]*merge.iloc[i,2]) - average_price)/average_price
        gains_without_currency = ((merge.iloc[i,1]) - average_price_without_currency)/average_price_without_currency
        roi_brutto_closed.append(gains)
    return roi_brutto_closed

In [44]:
def profit_current(merge, stock_degiro,for_zero):
    amount_of_bought_stocks = 0
    amount_of_sold_stocks = 0
    sum_of_bought_stocks = 0
    sum_of_sold_stocks = 0
    sum_of_bought_stocks_without_currency = 0
    sum_of_sold_stocks_without_currency = 0
    opłaty_trans = 0
    n = []
    roi_brutto_current = [0]*for_zero
    for i in range (len(merge)):      
        l = len(stock_degiro[stock_degiro['Data'].dt.date == merge.iloc[i][0].date()])
        if l != 0:
            for j in range(l):
                stock_deg = stock_degiro[stock_degiro['Data'].dt.date == merge.iloc[i][0].date()].reset_index(drop=True)
                if  stock_deg.iloc[j,6] > 0:
                    amount_of_bought_stocks += stock_deg.iloc[j,6]
                    sum_of_bought_stocks += (-1)*stock_deg.iloc[j,11]
                    sum_of_bought_stocks_without_currency += stock_deg.iloc[j,7]
                    a = -stock_deg.iloc[j,11]/stock_deg.iloc[j,6]
                    opłaty_trans += (-1)*stock_deg.iloc[j,14] 

                for k in range(stock_deg.iloc[j,6]):
                    n.append(a)

                average_price = sum_of_bought_stocks/amount_of_bought_stocks   
                average_price_without_currency = sum_of_bought_stocks_without_currency / amount_of_bought_stocks

                if  stock_deg.iloc[j,6] < 0:
                    amount_of_sold_stocks += stock_deg.iloc[j,6]
                    sum_of_sold_stocks = stock_deg.iloc[j,11]
                    sum_of_sold_stocks_without_currency = stock_deg.iloc[j,7]
                    opłaty_trans += (-1)*stock_deg.iloc[j,14]
                    for m in range((-1)*stock_deg.iloc[j,6]):
                        n.pop(0)
                    average_price = sum(n)/len(n)

        gains = ((merge.iloc[i,1]*merge.iloc[i,2]) - average_price)/average_price
        gains_without_currency = ((merge.iloc[i,1]) - average_price_without_currency)/average_price_without_currency
        infl_of_currency = gains - gains_without_currency
        roi_brutto_current.append(gains)
    return roi_brutto_current

In [45]:
def profit(merge, stock_degiro,for_zero):
    amount_of_bought_stocks = 0
    amount_of_sold_stocks = 0
    sum_of_bought_stocks = 0
    sum_of_sold_stocks = 0
    sum_of_bought_stocks_without_currency = 0
    sum_of_sold_stocks_without_currency = 0
    opłaty_trans = 0
    n = []
    roi_brutto_current = [0]*for_zero
    for i in range (len(merge)):      
        l = len(stock_degiro[stock_degiro['Data'].dt.date == merge.iloc[i][0].date()])
        if l != 0:
            for j in range(l):
                stock_deg = stock_degiro[stock_degiro['Data'].dt.date == merge.iloc[i][0].date()].reset_index(drop=True)
                if  stock_deg.iloc[j,6] > 0:
                    amount_of_bought_stocks += stock_deg.iloc[j,6]
                    sum_of_bought_stocks += (-1)*stock_deg.iloc[j,11]
                    sum_of_bought_stocks_without_currency += stock_deg.iloc[j,7]
                    a = -stock_deg.iloc[j,11]/stock_deg.iloc[j,6]
                    opłaty_trans += (-1)*stock_deg.iloc[j,14] 

                for k in range(stock_deg.iloc[j,6]):
                    n.append(a)

                average_price = sum_of_bought_stocks/amount_of_bought_stocks   
                average_price_without_currency = sum_of_bought_stocks_without_currency / amount_of_bought_stocks

                if  stock_deg.iloc[j,6] < 0:
                    amount_of_sold_stocks += stock_deg.iloc[j,6]
                    sum_of_sold_stocks = stock_deg.iloc[j,11]
                    sum_of_sold_stocks_without_currency = stock_deg.iloc[j,7]
                    opłaty_trans += (-1)*stock_deg.iloc[j,14]
                    for m in range((-1)*stock_deg.iloc[j,6]):
                        n.pop(0)
                    average_price = sum(n)/len(n)

        gains = ((merge.iloc[i,1]*merge.iloc[i,2]) - average_price)/average_price
        gains_without_currency = ((merge.iloc[i,1]) - average_price_without_currency)/average_price_without_currency
        infl_of_currency = gains - gains_without_currency
        roi_brutto_current.append(gains)
    return roi_brutto_current

In [46]:
keys_cur = list(s_current.keys())

NameError: name 's_current' is not defined

In [47]:
start = time.time()
k = 0
start_point = str(int(Transactions['Data'][0].replace(hour=0, minute=0, second=0, microsecond=0).timestamp()))
period2 = str(int(datetime.now().timestamp()))
fx = parse_currency(d_current['pair_currency'][k],start_point,period2)
roi_brutto_current = {}
for key in s:
    stock_degiro = s_current[key]
    stock_degiro['Data'] = pd.to_datetime(stock_degiro['Data'],dayfirst=True)
    stock_degiro = stock_degiro.sort_values('Data')
    if stock_degiro.iloc[0][11] == 0:
        continue
    period1 = str(int(stock_degiro.iloc[0][0].timestamp()))
    data_stock = parse_stock(key,period1,period2)
    if key == keys_cur[0]:
        for_zer = len(data_stock)
    for_zero = for_zer - len(data_stock)
    if d_current['pair_currency'][k] != 'EUR=X':
        fx = parse_currency(d_current['pair_currency'][k],period1,period2)
    
    merge = pd.merge(data_stock,fx,how='inner', on='Data').drop(['Open_x','High_x','Low_x','Close_x','Volume_x','Open_y','High_y','Low_y','Close_y','Volume_y'],axis=1)
    merge = merge.rename(columns = {'Data':'Date','Adj Close_x':'Stock_adj_close','Adj Close_y':'curr_adj_close'})

    roi_brutto_current [key] = profit(merge, stock_degiro,for_zero)
    k += 1
end = time.time()

TypeError: 'numpy.int64' object is not iterable

In [48]:
start = time.time()
k = 0
start_point = str(int(Transactions['Data'][0].replace(hour=0, minute=0, second=0, microsecond=0).timestamp()))
period2 = str(int(datetime.now().timestamp()))
fx = parse_currency(d_current['pair_currency'][k],start_point,period2)
roi_brutto_current = {}
for key in s_current:
    stock_degiro = s_current[key]
    stock_degiro['Data'] = pd.to_datetime(stock_degiro['Data'],dayfirst=True)
    stock_degiro = stock_degiro.sort_values('Data')
    if stock_degiro.iloc[0][11] == 0:
        continue
    period1 = str(int(stock_degiro.iloc[0][0].timestamp()))
    data_stock = parse_stock(key,period1,period2)
    if key == keys_cur[0]:
        for_zer = len(data_stock)
    for_zero = for_zer - len(data_stock)
    if d_current['pair_currency'][k] != 'EUR=X':
        fx = parse_currency(d_current['pair_currency'][k],period1,period2)
    
    merge = pd.merge(data_stock,fx,how='inner', on='Data').drop(['Open_x','High_x','Low_x','Close_x','Volume_x','Open_y','High_y','Low_y','Close_y','Volume_y'],axis=1)
    merge = merge.rename(columns = {'Data':'Date','Adj Close_x':'Stock_adj_close','Adj Close_y':'curr_adj_close'})

    roi_brutto_current [key] = profit_current(merge, stock_degiro,for_zero)
    k += 1
end = time.time()

NameError: name 's_current' is not defined

In [49]:
d_cur = pd.DataFrame.from_dict(roi_brutto_current)

In [50]:
d_cl = pd.DataFrame.from_dict(roi_brutto_closed)

NameError: name 'roi_brutto_closed' is not defined

In [51]:
keys_cl = list(s_closed.keys())

NameError: name 's_closed' is not defined

In [52]:
start = time.time()
k = 0
start_point = str(int(Transactions['Data'][0].replace(hour=0, minute=0, second=0, microsecond=0).timestamp()))
period2 = str(int(datetime.now().timestamp()))
fx = parse_currency(d_current['pair_currency'][k],start_point,period2)
roi_brutto_closed = {}
for key in s_closed:
    stock_degiro = s_closed[key]
    stock_degiro['Data'] = pd.to_datetime(stock_degiro['Data'],dayfirst=True)
    stock_degiro = stock_degiro.sort_values('Data')
    period1 = str(int(stock_degiro.iloc[0][0].timestamp()))
    period2 = str(int(stock_degiro.iloc[-1][0].timestamp()))
    data_stock = parse_stock(key,period1,period2)
    if key == keys_cl[0]:
        for_zer = len(data_stock)
    for_zero = for_zer - len(data_stock)
    if d_closed['pair_currency'][k] != 'EUR=X':
        fx = parse_currency(d_current['pair_currency'][k],period1,period2)
    
    merge = pd.merge(data_stock,fx,how='inner', on='Data').drop(['Open_x','High_x','Low_x','Close_x','Volume_x','Open_y','High_y','Low_y','Close_y','Volume_y'],axis=1)
    merge = merge.rename(columns = {'Data':'Date','Adj Close_x':'Stock_adj_close','Adj Close_y':'curr_adj_close'})

    roi_brutto_closed [key] = profit_closed(merge, stock_degiro,for_zero)
    k += 1
end = time.time()

NameError: name 's_closed' is not defined

In [53]:
s_closed = {}
for i in range (len(d_closed)):
    degiro_name = d_closed['degiro_name'][i]
    s_closed[d_closed['ticker'][i]] = closed_pos[closed_pos['Produkt'] == degiro_name]

NameError: name 'd_closed' is not defined

In [54]:
s_current = {}
for i in range (len(d_current)):
    degiro_name = d_current['degiro_name'][i]
    s_current[d_current['ticker'][i]] = current_pos[current_pos['Produkt'] == degiro_name]

In [55]:
s = {}
l = len(d_closed) + len(d_current)

# for i in range (len(d_current)):
#     degiro_name = d_current['degiro_name'][i]
#     s[d_current['ticker'][i]] = current_pos[current_pos['Produkt'] == degiro_name]
    
for j in range (l - len(d_current)):
    degiro_name = d_closed['degiro_name'][j]
    s[d_closed['ticker'][j]].append(closed_pos[closed_pos['Produkt'] == degiro_name])

NameError: name 'd_closed' is not defined

In [56]:
s.keys()

dict_keys([])

In [57]:
account = pd.read_csv('Account.csv')

In [58]:
account = account.rename({'Unnamed: 8': 'wartość', 'Unnamed: 10': 'saldo'},axis= 1)

In [59]:
account['wartość'] = account.wartość.str.replace(',','.').astype(float)

In [60]:
tax_div = account[account['Opis'] == 'Podatek Dywidendowy'].reset_index(drop=True)

In [61]:
tax_trans = account.loc[account['Opis'].str.contains('od transakcji', case = False, na = False)].reset_index(drop=True)

In [62]:
div = account[account['Opis'] == 'Dywidenda']
div = div.reset_index(drop=True)

In [63]:
stocks = div['Produkt'].unique()

In [64]:
d = {}
for i in range (len(stocks)):
    s = 0
    for j in range (len(div)):
        if stocks[i] == div['Produkt'][j]:
            s += div['wartość'][j]
    d[stocks[i]] = s
    
            

In [65]:
t_d = {}
for i in range(len(stocks)):
    t = 0
    for j in range(len(tax_div)):
        if stocks[i] == tax_div['Produkt'][j]:
            t += tax_div['wartość'][j]
    t_d[stocks[i]] = t

In [66]:
netto_div = {}
for key in (d):
    n = d[key] - (-t_d[key])
    netto_div[key] = n

In [67]:
t_t = {}
for i in range(len(stocks)):
    tt = 0
    for j in range(len(tax_trans)):
        if stocks[i] == tax_trans['Produkt'][j]:
            tt += tax_trans['wartość'][j]
    t_t[stocks[i]] = tt

In [68]:
period1 = str(int(Transactions['Data'][0].replace(hour=0, minute=0, second=0, microsecond=0).timestamp()))
period2 = str(int(datetime.now().timestamp()))

In [69]:
sp = parse_stock('^GSPC',period1,period2 ).reset_index(drop=True)
sp['Open'] = pd.to_numeric(sp['Open'])

In [70]:
sp['weight'] = None
sp['weight'][0] = sp.iloc[0][5]/sp.iloc[0][1]
for i in range(1,len(sp)):
    sp['weight'][i] = sp.iloc[i][5]/sp.iloc[i-1][5]

In [71]:
sp['portfolio'] = None
sp['portfolio_change'] = None
sp['top up'] = None

sp_portfolio = (-1)*Transactions.iloc[0,9]*sp['weight'][0]
sp['portfolio'][0] = sp_portfolio
s = (-1)*Transactions.iloc[0,9]
sp['top up'][0] = s
sp['portfolio_change'][0] = ((sp_portfolio - s)/s)*100

for i in range (1,len(sp)):
    l = len(Transactions[Transactions['Data'] == sp.iloc[i,0]])
    if l == 0:
        sp_portfolio = sp_portfolio* sp['weight'][i]
    else:
        for j in range(l):
            Transact = Transactions[Transactions['Data'] == sp.iloc[i,0]].reset_index(drop=True)
            s += (-1)*Transact.iloc[j,9]
            sp_portfolio += (-1)*Transact.iloc[j,9]  
        sp_portfolio = sp_portfolio* sp['weight'][i]
        portfolio_change = ((sp_portfolio - s)/s)*100
    sp['portfolio'][i] = sp_portfolio
    sp['top up'][i] = s
    sp['portfolio_change'][i] = ((sp_portfolio - s)/s)*100